In [226]:
import numpy as np
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Softmax


In [187]:
with open('data/shakespeare_proc_rnn.txt', 'r') as f:
    source = f.read()

char_seqs = [source[i-40:i+1] for i in range(40, len(source))]
char_to_int = {}
int_to_char = {}
for i, char in enumerate(set(source)):
    char_to_int[char] = i
    int_to_char[i] = char
    
num_chars = len(char_to_int)
encoded = np.array([[char_to_int[char] for char in seq] for seq in char_seqs])

In [8]:
X = np.array([to_categorical(c, num_classes=num_chars) for c in encoded[:,:-1]])
y = to_categorical(encoded[:,-1], num_classes=num_chars)
    
model = Sequential()
model.add(LSTM(150, input_shape=(X.shape[1], X.shape[2])))
model.add(Dense(num_chars, activation='softmax'))
model.compile('adam', loss='categorical_crossentropy')
model.fit(X, y, epochs=50)
model.save('rnn_model')

Epoch 1/50
93786/93786 [==============================] - 80s 857us/step - loss: 2.3191
Epoch 2/50
93786/93786 [==============================] - 79s 848us/step - loss: 1.9219
Epoch 3/50
93786/93786 [==============================] - 77s 817us/step - loss: 1.7793
Epoch 4/50
93786/93786 [==============================] - 79s 841us/step - loss: 1.6902
Epoch 5/50
93786/93786 [==============================] - 78s 827us/step - loss: 1.6235
Epoch 6/50
93786/93786 [==============================] - 81s 869us/step - loss: 1.5690
Epoch 7/50
93786/93786 [==============================] - 81s 861us/step - loss: 1.5207
Epoch 8/50
93786/93786 [==============================] - 77s 824us/step - loss: 1.4769
Epoch 9/50
93786/93786 [==============================] - 83s 882us/step - loss: 1.4353
Epoch 10/50
93786/93786 [==============================] - 78s 833us/step - loss: 1.3973
Epoch 11/50
93786/93786 [==============================] - 79s 844us/step - loss: 1.3606
Epoch 12/50
93786/93786 [=====

In [ ]:
X = np.array([to_categorical(c, num_classes=num_chars) for c in encoded[:,:-1]])
y = to_categorical(encoded[:,-1], num_classes=num_chars)
    
model = Sequential()
model.add(LSTM(150, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(LSTM(50, input_shape=(X.shape[1], X.shape[2])))
model.add(Dense(num_chars, activation='softmax'))
model.compile('adam', loss='categorical_crossentropy')
model.fit(X, y, epochs=50)
model.save('deep_rnn_model')

In [197]:
model = load_model('rnn_model')

temperature = 0.25

text = 'shall i compare thee to a summer\'s day?\n'
for i in range(600):
    encoded = [char_to_int[char] for char in text]
    encoded = pad_sequences([encoded], maxlen=40, truncating='pre')
    encoded = to_categorical(encoded, num_classes=num_chars)
    pred = model.predict(encoded, verbose=0)
    pred = np.exp(np.log(pred[0]) / temperature)
    pred /= (np.sum(pred) + .00001)
    output = np.argmax(np.random.multinomial(1, pred, 1))
    text += int_to_char[output]

print(text)

shall i compare thee to a summer's day?
that in the breath their victurn true,
and therefore with thou art poart to my sight,
where thou wert thou wilt, thou art plea trust live,
my self ink in love that to blest eyes.
that they see aprieving to the world to conspaice.
but that tongue that i have some in their show,
they doy presate that thou shouldst in men's love,
when i am self in summer's not cone,
which thou art trust before hath my true,
still thou feeding and heart doth can despaised,
even in the will sweet self appear where.
in every new lose posterity the tweet,
the clocked it in me that thou shouldst ploved.
i till the gra


In [225]:
model = load_model('deep_rnn_model')

temperature = 0.25

text = 'shall i compare thee to a summer\'s day?\n'
for i in range(582):
    encoded = [char_to_int[char] for char in text]
    encoded = pad_sequences([encoded], maxlen=40, truncating='pre')
    encoded = to_categorical(encoded, num_classes=num_chars)
    pred = model.predict(encoded, verbose=0)
    pred = np.exp(np.log(pred[0]) / temperature)
    pred /= (np.sum(pred) + .00001)
    output = np.argmax(np.random.multinomial(1, pred, 1))
    text += int_to_char[output]

print(text)

shall i compare thee to a summer's day?
then beauteous shave no wert thou thy soul,
i see thing a shadetre that my and thine.
that thou gaves, that thy sweet self too,
in such a self-love and strangely, and thee,
they do i none doth her sight, and dost show,
sweet think on thines to his sun then marged,
when in the wantard thou gild'st the worn,
and think in me thou desire im received
for their excelled that like a summer's stay
a sonly new on that love's fingers winten
bearing the times are not seem do the day
steal thy heart that which thou thy self dost seeing,
and thoughts to the table is never make shows,
and 
